### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [8]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [17]:
num_iterations = 1
empty_list = []
for i in range(12, 17):
    if i == 8:
        continue
    search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    web_format = g_filter[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.28399082758168637
cusum = 0.05721359056243869
eta = 2.3179301765464833
hl_amp_ratio = 0.4024947034892876
kurtosis = -0.2626460675749649
n_points = 451
period = 0.540405033280329
period_SNR = 115.06380590559186
period_log10FAP = -57.39012655416996
period_uncertainty = 0.00014296430452420683
phase_cusum = 0.23490460808150543
phase_eta = 0.4251584205916919
phi21 = 0.8297619100693845
phi31 = 1.91741478690491
quartile31 = 0.34665867146890506
r21 = 0.41897375818170934
r31 = 0.24012230871245
shapiro_w = 0.94781494140625
skewness = -0.6965091882910406
slope_per10 = -0.060459943459194004
slope_per90 = 0.0902216851322205
stetson_k = 0.7478114049210987
weighted_mean = 14.216313977039217
weighted_std = 0.22618426595576677
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.98


######### Lightcurve features #########

amplitude = 0.13949436751359698
cusum = 0.09405608411483704
eta = 2.175508647526293
hl_amp_ratio = 1.023674438650023
kurtosis = -0.7610700567735802
n_points = 1112
period = 0.30550999910812254
period_SNR = 165.28877359680823
period_log10FAP = -149.79649406498154
period_uncertainty = 3.7637363075826435e-05
phase_cusum = 0.2576835529125052
phase_eta = 0.7755863240103243
phi21 = -1.6608479417108013
phi31 = -3.563786265080089
quartile31 = 0.20472368184754863
r21 = 0.11832084500575656
r31 = 0.044377664152432564
shapiro_w = 0.9852181077003479
skewness = 0.006756277837516903
slope_per10 = -0.020843532481369536
slope_per90 = 0.02059080156335378
stetson_k = 0.8309428391911695
weighted_mean = 15.2316065738242
weighted_std = 0.11634583046038519
Pulled 1 of 1


...

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.43993733205701935
cusum = 0.2625617858776915
eta = 1.3928703464770895
hl_amp_ratio = 0.3317130422850503
kurtosis = -0.5182641703589765
n_points = 2239
period = 0.5058775936653525
period_SNR = 279.6739537097334
period_log10FAP = -304.9741267826213
period_uncertainty = 0.00011050188243838521
phase_cusum = 0.3016755478315001
phase_eta = 0.2418821666624599
phi21 = -2.514216710482801
phi31 = -1.5184212916458777
quartile31 = 0.5768474642988402
r21 = 0.4580569762868193
r31 = 0.32801657742915796
shapiro_w = 0.9114194512367249
skewness = -0.7539927709585739
slope_per10 = -0.013640821401202749
slope_per90 = 0.012808683114646701
stetson_k = 0.6518506226251092
weighted_mean = 15.118964885719265
weighted_std = 0.39454781928057175
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_d
class_probability = 0.89


######### Lightcurve features #########

amplitude = 0.21275303129629525
cusum = 0.16662252610455353
eta = 2.0353270957699956
hl_amp_ratio = 0.8861886720987744
kurtosis = -0.4455625300518795
n_points = 1687
period = 0.3687396416634942
period_SNR = 226.26535962950194
period_log10FAP = -192.38287793219476
period_uncertainty = 4.9375957151875927e-05
phase_cusum = 0.30405948814035394
phase_eta = 1.0334907304886
phi21 = 1.4581181044760132
phi31 = -0.763898772342307
quartile31 = 0.30173227760721133
r21 = 0.1672266140178455
r31 = 0.06458762383480723
shapiro_w = 0.9933198094367981
skewness = -0.07470608485574937
slope_per10 = -0.010278773126172536
slope_per90 = 0.0096140350875362
stetson_k = 0.7680697216831229
weighted_mean = 15.677961976962461
weighted_std = 0.1869816238913165


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
